# Example of creating data, preprocess with noise and train statistical models
In this example we will train statistical models on node classification. To this end, we will first generate a transaction log with Swish-AMLsim. Then split the data into two parts: a trainset and a testset. The split is done in time with a overlap. For each set we then build a graph of nodes and edges. The nodes represent the bank accounts and the edges represent reletions between two accounts. The nodes and edges will have aggregated features of the transactions and the labels will be on the nodes. The label is negative (0) if the node didn't participate in money laundering acitivities and positive (1) otherwise. 

The features on the nodes are:
- sums, means, medians, stds, maxs, mins, counts of spending transactions (transactions to the sink).
- sums, means, medians, stds, maxs, mins of swish transactions (transactions within the network).
- number of incoming and outgoing swish transactions.
- number of unique accounts the node has sent to and recived by.

The features of the edges are:
- sums, means, medians, stds, maxs, mins, counts of transactions between the two nodes.

The labels are:
- 0 if the node didn't participate in money laundering acitivities.
- 1 if the node participated in money laundering acitivities. 

After building the graph, we will add noise to the labels in different fashion. In this example, there are four types of noise:
- Flipped labels: random nodes flip their labels.
- Misssing labels: random nodes will lose their label.
- Neighbors noise: random negative nodes will have their labels flipped if they have a positive neighbor.
- Topological noise: random nodes in specific topologies will have their labels flipped, suggesting a bank missing labels for specific money laundering stratergies.
We will create four different datasets, one for each type of noise.

Finally, we will train statistical models on the noisy data and evaluate the performance of the model on the test set. The models are:
- Logistic regression
- Random forest
- Gradient boosting
- Support vector machine
- K-nearest neighbors

The result are TN, FP, FN and TP for 101 threshold between 0 and 1. These can be used calculate wanted matics like ROC, AUC, F1, precision, recall, and accuracy.

In [ ]:
# imports
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from preprocess.feature_engineering import *
from preprocess.noise import *
from benchmark.stat_models import *

## Create data
TODO: write code to run AMLsim from a notebook.

For now we will use a pre-generated dataset. 

In [ ]:
dataset_name = '10K_accts'
dataset_path = f'../AMLsim/outputs/10K_accts/tx_log.csv'

## Preprocess data


In [ ]:
df = load_data(dataset_path)
bank = 'bank'
overlap = 0.5 # overlap of training and testing data
windows = [(0, 30)] # int or list of tuples - if int then the number of windows, if list of tuples then the start and end step for each window

df_bank = df[(df['bankOrig'] == bank) | (df['bankDest'] == bank)]

train_start = df_bank['step'].min()
train_end = df_bank['step'].min() + (df_bank['step'].max() - df_bank['step'].min()) * (overlap+(1-overlap)/2)

test_start = df_bank['step'].min() + (df_bank['step'].max() - df_bank['step'].min()) * (1-overlap)/2
test_end = df_bank['step'].max()

df_bank_train = df_bank[(df_bank['step'] >= train_start) & (df_bank['step'] <= train_end)]
df_bank_test = df_bank[(df_bank['step'] >= test_start) & (df_bank['step'] <= test_end)]

df_nodes_train = get_nodes(df_bank_train, bank, windows)
df_edges_train = get_edges(df_bank_train[(df_bank_train['bankOrig'] == bank) & (df_bank_train['bankDest'] == bank)], windows, aggregated=True, directional=True) # TODO: enable edges to/from the bank? the node features use these txs but unclear how to ceate a edge in this case, the edge can't be connected to a node with node features (could create node features based on edge txs, then the node features and edge features will look the same and some node features will be missing)

df_nodes_test = get_nodes(df_bank_test, bank, windows)
df_edges_test = get_edges(df_bank_test[(df_bank_test['bankOrig'] == bank) & (df_bank_test['bankDest'] == bank)], windows, aggregated=True, directional=True)

df_nodes_train.reset_index(inplace=True)
node_to_index = pd.Series(df_nodes_train.index, index=df_nodes_train['account']).to_dict()

df_edges_train['src'] = df_edges_train['src'].map(node_to_index) # OBS: in the csv files it looks like the edge src refers to the node two rows above the acculat node, this is due to the column head and that it starts counting at 0
df_edges_train['dst'] = df_edges_train['dst'].map(node_to_index)

df_nodes_test.reset_index(inplace=True)
node_to_index = pd.Series(df_nodes_test.index, index=df_nodes_test['account']).to_dict()

df_edges_test['src'] = df_edges_test['src'].map(node_to_index)
df_edges_test['dst'] = df_edges_test['dst'].map(node_to_index)

print('\ntrainset:\nnodes:')
display(df_nodes_train.head())
print('edges:')
display(df_edges_train.head())
print('\ntestset:\nnodes:')
display(df_nodes_test.head())
print('edges:')
display(df_edges_test.head())



## Add noise to the labels

### Flipped labels


In [ ]:
labels = [0, 1] # labels that will be affected by noise
fracs = [0.01, 0.1] # fractions of nodes that will be affected by noise for each label
df_flipped_lables = flip_labels(df_nodes_train, labels, fracs)
display(df_flipped_lables.head())

### Missing labels

In [ ]:
labels = [0, 1] # labels that will be affected by noise
fracs = [0.01, 0.1] # fractions of nodes that will be affected by noise for each label
df_missing_labels = missing_labels(df_nodes_train, labels, fracs)
display(df_missing_labels.head())

### Neighbour noise

In [ ]:
neighbour_frac = 0.5 # fraction of negative neighbours that will be flipped to positive
df_flipped_neighbours = flip_neighbours(df_nodes_train, df_edges_train, neighbour_frac)
display(df_flipped_neighbours.head())

### Topology noise
We will add noise to 'gather_scatter', 'scatter_gather', 'stack' while keeping fan_in, fan_out and bipartite untoched to see if the models can generalize to more complex patterns.

In [ ]:
topologies = ['gather_scatter', 'scatter_gather', 'stack'] 
topology_frac = 0.5 # fraction of labels in the topologies that will be affected by noise

# to add the topology noise we need to know the alert members, this will be changed in the future
alert_members = pd.read_csv(dataset_path.replace('outputs', 'tmp').replace('tx_log.csv', 'alert_members.csv'))

df_flipped_topologies = topology_noise(df_nodes_train, alert_members, topologies, fracs[1])
display(df_flipped_topologies.head())

## Train models on the datasets


In [ ]:
model_names = ['XGB','RF','SVM','KNN','LOG']
thresholds = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]

In [ ]:

datasets = (df_nodes_train, df_nodes_test)
results = train_models(datasets, model_names)
for model_name, metrics in results.items():
    print(model_name)
    for metric, value in metrics.items():
        if metric in thresholds:
            print(f'  threshold: {metric}: {value}')

In [ ]:
dataset = (df_flipped_lables, df_nodes_test) 
results = train_models(datasets, model_names)
for model_name, metrics in results.items():
    print(model_name)
    for metric, value in metrics.items():
        if metric in thresholds:
            print(f'  threshold: {metric}: {value}')

In [ ]:
dataset = (df_missing_labels, df_nodes_test) 
results = train_models(datasets, model_names)
for model_name, metrics in results.items():
    print(model_name)
    for metric, value in metrics.items():
        if metric in thresholds:
            print(f'  threshold: {metric}: {value}')

In [ ]:
dataset = (df_flipped_neighbours, df_nodes_test)
results = train_models(datasets, model_names)
for model_name, metrics in results.items():
    print(model_name)
    for metric, value in metrics.items():
        if metric in thresholds:
            print(f'  threshold: {metric}: {value}')

In [ ]:
dataset = (df_flipped_topologies, df_nodes_test)
results = train_models(datasets, model_names)
for model_name, metrics in results.items():
    print(model_name)
    for metric, value in metrics.items():
        if metric in thresholds:
            print(f'  threshold: {metric}: {value}')